In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
import keras
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/jigsaw-toxic-comment-classification-challenge/test.csv.zip
/kaggle/input/jigsaw-toxic-comment-classification-challenge/train.csv.zip
/kaggle/input/jigsaw-toxic-comment-classification-challenge/sample_submission.csv.zip
/kaggle/input/jigsaw-toxic-comment-classification-challenge/test_labels.csv.zip


Using TensorFlow backend.


In [2]:
df=pd.read_csv('/kaggle/input/jigsaw-toxic-comment-classification-challenge/train.csv.zip')
df.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [3]:
import matplotlib.pyplot as plt
import re
import nltk

In [4]:
X_train=df.comment_text
Y_train=df.drop(['id','comment_text'],axis=1)

In [5]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation, GRU
from keras.layers import Bidirectional, GlobalMaxPool1D
from keras.models import Model
from keras import initializers, regularizers, constraints, optimizers, layers

In [6]:
max_words=20000
tokenizer=Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(list(X_train))

In [7]:
def preprocess_textdata(text_array,pad_length=200):
    token_txt=tokenizer.texts_to_sequences(text_array)
    padded_txt=pad_sequences(token_txt,maxlen=pad_length)
    return padded_txt

In [8]:
padded_text=preprocess_textdata(X_train,200)

In [24]:
padded_text.shape

(159571, 200)

In [9]:
from sklearn.model_selection import train_test_split as tts
x_train,x_val,y_train,y_val=tts(padded_text,Y_train,test_size=0.1,random_state=10)

## Using Bi directional LSTM

In [10]:
keras.backend.clear_session()
inp=Input(shape=(200,))
embed_size=128
X=Embedding(max_words,embed_size)(inp)
X=Bidirectional(LSTM(60,return_sequences=True))(X)
#X=LSTM(60,return_sequences=True)(X)
X=GlobalMaxPool1D()(X)
X=Dropout(0.1)(X)
X=Dense(60,activation='relu')(X)
X=Dropout(0.1)(X)
X=Dense(6,activation='sigmoid')(X)
model=Model(inputs=inp, outputs=X)
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 200)               0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 200, 128)          2560000   
_________________________________________________________________
bidirectional_1 (Bidirection (None, 200, 120)          90720     
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 120)               0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 120)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 60)                7260      
_________________________________________________________________
dropout_2 (Dropout)          (None, 60)                0   

In [11]:
model.fit(x_train,y_train,batch_size=512,epochs=2)

/opt/conda/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/2
143613/143613 [==============================] - 105s 734us/step - loss: 0.1302 - accuracy: 0.9639
Epoch 2/2
143613/143613 [==============================] - 102s 710us/step - loss: 0.0530 - accuracy: 0.9812


In [12]:
model.evaluate(x_val,y_val,batch_size=512)

15958/15958 [==============================] - 2s 132us/step


[0.051015271758421396, 0.982380747795105]

## Replacing LSTM with GRU layer

In [13]:
inp=Input(shape=(200,))
embed_size=128
X=Embedding(max_words,embed_size)(inp)
#X=Bidirectional(LSTM(60,return_sequences=True))(X)
X=GRU(120,return_sequences=True)(X)
X=GlobalMaxPool1D()(X)
X=Dropout(0.1)(X)
X=Dense(60,activation='relu',kernel_initializer='glorot_uniform')(X)
X=Dropout(0.1)(X)
X=Dense(6,activation='sigmoid',kernel_initializer='glorot_uniform')(X)
model2=Model(inputs=inp, outputs=X)
model2.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
model2.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 200)               0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 200, 128)          2560000   
_________________________________________________________________
gru_1 (GRU)                  (None, 200, 120)          89640     
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 120)               0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 120)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 60)                7260      
_________________________________________________________________
dropout_2 (Dropout)          (None, 60)                0   

In [14]:
model2.fit(x_train,y_train,batch_size=512,epochs=3)

/opt/conda/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/3
143613/143613 [==============================] - 68s 474us/step - loss: 0.1688 - accuracy: 0.9595
Epoch 2/3
143613/143613 [==============================] - 68s 473us/step - loss: 0.0767 - accuracy: 0.9749
Epoch 3/3
143613/143613 [==============================] - 68s 475us/step - loss: 0.0473 - accuracy: 0.9824


In [15]:
model2.evaluate(x_val,y_val,batch_size=512)

15958/15958 [==============================] - 1s 84us/step


[0.04808481595966732, 0.982339084148407]

In [16]:
testing_df=pd.read_csv('/kaggle/input/jigsaw-toxic-comment-classification-challenge/test.csv.zip')
testing_df.head()

,id,comment_text
0,00001cee341fdb12,Yo bitch Ja Rule is more succesful then you'll...
1,0000247867823ef7,== From RfC == \n\n The title is fine as it is...
2,00013b17ad220c46,""" \n\n == Sources == \n\n * Zawe Ashton on Lap..."
3,00017563c3f7919a,":If you have a look back at the source, the in..."
4,00017695ad8997eb,I don't anonymously edit articles at all.


In [17]:
x_test=testing_df['comment_text']
padded_test=preprocess_textdata(x_test)

In [30]:
y_test=model2.predict(padded_test,batch_size=512)

In [27]:
testing_df['toxic']=y_test[:,0]
testing_df['severe_toxic']=y_test[:,1]
testing_df['obscene']=y_test[:,2]
testing_df['threat']=y_test[:,3]
testing_df['insult']=y_test[:,4]
testing_df['identity_hate']=y_test[:,5]
testing_df.drop('comment_text',axis=1,inplace=True)
testing_df.head()

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,00001cee341fdb12,0.995331,0.407550,0.952756,0.053716,0.884706,0.153889
1,0000247867823ef7,0.001353,0.000010,0.000173,0.000021,0.000261,0.000061
2,00013b17ad220c46,0.001979,0.000018,0.000247,0.000037,0.000411,0.000097
3,00017563c3f7919a,0.001354,0.000004,0.000107,0.000010,0.000162,0.000034
4,00017695ad8997eb,0.004180,0.000045,0.000551,0.000094,0.000858,0.000247


In [28]:
testing_df.to_csv('/kaggle/working/out.csv',index=None)